# An analysis of the influence of Black Southern Churches on the Southern Black Community

## Background:

Documenting the American South is one of the longest running digital publishing initiatives at the University of North Carolina. It was designed to give researchers digital access to some of the library’s unique collections in the form of high quality page scans as well as structured, corrected and machine readable text. (https://docsouth.unc.edu/docsouthdata/)

## Goal: 

Analyze rhetoric of the Black Southern Church in the American South and it's effects on documents written by self emancipated and previously enslaved Black people.

## Research question:

Are there any measurable similaries between the themes of documents from Southern Black churches and the documents from self-emancipated and freed Black people? 

## Approach:

[Documenting The American South](https://docsouth.unc.edu/) is one of the longest running efforts by the University of North Carolina to collect, digitize, and publish documents from self-emancipated and freed Black people. Using [DocSouth Data](https://docsouth.unc.edu/docsouthdata/) and data from the [Religious Text Content Guide](https://docsouth.unc.edu/neh/religiouscontent.html), analyze the correlation between autobiographies, biographies, church documents, sermons, histories, encyclopedias, and other published materials from Southern Black churches and the narratives of slaves in regards to religion.

Proposed analysis:

- Evaluate language of both data sets
- Evaluate themes of both data sets
- Using data from the [Religious Text Content Guide](https://docsouth.unc.edu/neh/religiouscontent.html), develop a theme predictor for texts
- Evaluate how much (if any) thematic overlap there is between religious texts and slave narratives


data: 
https://www.kaggle.com/datasets/docsouth-data/the-church-in-the-southern-black-community
https://docsouth.unc.edu/neh/religiouscontent.html

note from kaggle compiler:
If you make use of this data, considering letting the holder of the original collection know how you are using the data and if you have any suggestions for making it even more useful. Send any feedback to wilsonlibrary@unc.edu.

In [1]:

#read in toc table
#replace .xml with .txt for filenames
# remove anything in [] from txt files
import pandas as pd
from pandas import DataFrame as df
from  config import CONFIG



toc_data = CONFIG["TOC_FILEPATH"]
contents_table = pd.read_csv(toc_data)
# TEXTS_FOLDER_PATH

txt_filename = contents_table.Filename.str.replace(".xml", ".txt")

# get every entry, drop .xml and replace it with .txt. Remember, strings are immutable
contents_table["txt_filename"] = txt_filename
# make new table
specified_content_cols = ["txt_filename", "Date", "Author", "Title"]
content_ref_table = contents_table[specified_content_cols]


/var/folders/18/918s5vr16kl9091zgnswr5gm0000gn/T/ipykernel_61433/3281841785.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  txt_filename = contents_table.Filename.str.replace(".xml", ".txt")


In [2]:
# rename date column and lower other col names
content_ref_table = content_ref_table.rename(columns={"Date":"year"})
content_ref_table.columns = content_ref_table.columns.str.lower()
content_ref_table

,txt_filename,year,author,title
0,church-ridgel-ridgel.txt,1896,Alfred Lee Ridgel,Africa and African Methodism
1,church-bowen-bowen.txt,1896,J. W. E. Bowen,Africa and the American Negro: Addresses and P...
2,neh-johnsontl-johnsontl.txt,1892,Thomas L. Johnson,Africa for Christ. Twenty-Eight Years a Slave
3,church-turneral-turner.txt,1893,Henry McNeal Turner,African Letters
4,church-gaineswj-gaines.txt,1890,W. J. Gaines,"African Methodism in the South; or, Twenty-Fiv..."
...,...,...,...,...
139,church-wheeler-wheeler.txt,1906,B. F. Wheeler,The Varick Family
140,church-cooper-cooper.txt,1892,Anna J. Cooper,A Voice from the South
141,neh-wilkerson-wilkerson.txt,1861,Major James Wilkerson,Wilkerson's History of His Travels &amp; Labor...
142,church-brawley-brawley.txt,c1919,Benjamin Griffith Brawley,Women of Achievement: Written for the Fireside...


In [3]:
# import data scraped from the web and turn it into a df
from scrape_website import scrape_data, custom_filter, build_dict
url = "https://docsouth.unc.edu/neh/religiouscontent.html"
tag_list = scrape_data(url=url, filter=custom_filter)
data_dict = build_dict(tag_list=tag_list)
content_guide_data = df.from_dict(data_dict)
content_guide_data


,theme,alt_theme,author,title,year,page
0,Baptism,None,"Brown, Henry Box, and Charles Stearn",Narrative of Henry Box Brown,1849,"(23-24, /neh/boxbrown/boxbrown.html#p23)"
1,Baptism,None,"Brown, Josephine",Biography of an American Bondman,"1856, c1855","(23-24, /neh/brownj/brownj.html#brownj23)"
2,Baptism,None,"Brown, Rebecca Warren[?]",Memoir of Mrs. Chloe Spear,1832,"(38-40, /neh/brownrw/brownrw.html#spear38)"
3,Baptism,None,"Henry, George",Life of George Henry,1894,"(42-43, /neh/henryg/henryg.html#henry42)"
4,Baptism,None,"Johnson, Thomas Lewis",Twenty-Eight Years a Slave,1909,"(17, /neh/johnson1/johnson.html#john17)"
...,...,...,...,...,...,...
390,Spirituals,None,"Jones, Friday",Days of Bondage,1883,"(17-18, /neh/fjones/jones.html#jones17)"
391,Spirituals,None,"Pickard, Kate E. R.",The Kidnapped and the Ransomed,1856,"(179-180, /neh/pickard/pickard.html#pickard179)"
392,Spirituals,None,"Smith, James Lindsay",Autobiography of James L. Smith,1881,"(27, /neh/smithj/smithj.html#jsmith27)"
393,[Conjure],None,"Hopkins, Dwight N.",Cut Loose Your Stammering Tongue: Black Theolo...,None,"(None, None)"


In [10]:
# get text files from 
base_url = "https://docsouth.unc.edu"

,theme,alt_theme,author,title,year,page
0,NaN,None,NaN,NaN,NaN,NaN
1,NaN,None,NaN,NaN,NaN,NaN
2,NaN,None,NaN,NaN,NaN,NaN
3,NaN,None,NaN,NaN,NaN,NaN
4,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
390,NaN,None,NaN,NaN,NaN,NaN
391,NaN,None,NaN,NaN,NaN,NaN
392,NaN,None,NaN,NaN,NaN,NaN
393,NaN,None,NaN,NaN,None,NaN
